Constance Delannoy

APPM 4650 | Homework 4

July 8, 2020

In [1]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import math

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="The raw code is by default hidden for easier reading.To toggle on/off the raw code, click here."></form>''')

__Question 1__

We have $\int_0^1 f(x)dx = af(\frac{1}{3}) + bf(\frac{2}{3})$

Let $t = 2x-1$ so that: \begin{cases} x=0 \rightarrow t = -1 \\ = x=1 \rightarrow t-1 \\ x = (t+1)/2\\ \frac{dt}{dx} = 2\end{cases}

The problem becomes: $\int_0^1 f(x)dx = \int_{-1}^1 2d(\frac{t+1}{2})dt$.

Using the direct method, we now get:

$\int_{-1}^1 2dt = 2x \rvert_{-1}^1 = 4 \rightarrow a+b = 4$

$\int_{-1}^1 2xdt = x^2 \rvert_{-1}^1 = 0 \rightarrow \frac{a}{3} + \frac{2b}{3} = 0 \rightarrow a = -2b$

Thus: $a=8$ and $b=-4$ so that:

$\int_0^1 f(x)dx = 8f(\frac{1}{3}) - 4f(\frac{2}{3})$

__Question 2__

In [80]:
def rk4(f1, f2, f1true, f2true, a, b, h):
    "applying RK-4 to system of coupled ODEs when we know the truth"
    
    res = {'x':[0], 'y1_est':[1], 'y2_est': [1], 'y1_error': [0], 'y2_error':[0], 'y1_true': [1], 'y2_true': [1]}
    steps = list(map(lambda x: x/10, range(0, 11, 2)))
    
    for t in steps:
        
        k11 = h*f1(t, a, b)
        k12 = h*f2(t, a, b)
        
        k21 = h*f1(t + h/2, a + k11/2, b + k12/2)
        k22 = h*f2(t + h/2, a + k11/2, b + k12/2)
        
        k31 = h*f1(t + h/2, a + k21/2, b + k22/2)
        k32 = h*f2(t + h/2, a + k21/2, b + k22/2)
        
        k41 = h*f1(t + h, a + k31, b + k32)
        k42 = h*f2(t + h, a + k31, b + k32)
        
        f1_est = a + (1/6)*(k11 + 2*k21 + 2*k31 + k41)
        f2_est = b + (1/6)*(k12 + 2*k22 + 2*k32 + k42)
        
        f1_true = f1true(t+h)
        f2_true = f2true(t+h)
        
        f1_err = abs(f1_true - f1_est)
        f2_err = abs(f2_true - f2_est)
        
        res['x'].append(t)
        res['y1_est'].append(f1_est)
        res['y2_est'].append(f2_est)
        res['y1_error'].append(f1_err)
        res['y2_error'].append(f2_err)
        res['y1_true'].append(f1_true)
        res['y2_true'].append(f2_true)
        
        a = f1_est
        b = f2_est
        
    return res

In [81]:
def fun1(t, a, b):
    "this is u'1 in the homework"

    y = 3*a + 2*b - (2*t**2 + 1)*math.exp(2*t)
    
    return y

def fun2(t, a, b):
    "this is u'2 in the homework"
    
    y = 4*a + b + (t**2 + 2*t - 4)*math.exp(2*t)
    
    return y

def fun1_true(t):
    "this is u1(t) in the homework"
    
    y = (1/3)*math.exp(5*t) - (1/3)*math.exp(-t) + math.exp(2*t)
    
    return y

def fun2_true(t):
    "this is u2(t) in the homework"
    
    y = (1/3)*math.exp(5*t) + (2/3)*math.exp(-t) + (t**2)*math.exp(2*t)
    
    return y

In [82]:
pd.DataFrame(rk4(fun1, fun2, fun1_true, fun2_true, 1, 1, 0.2))

,x,y1_error,y1_est,y1_true,y2_error,y2_est,y2_true
0,0.0,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
1,0.0,0.004643,2.120366,2.125008,0.004596,1.506992,1.511587
2,0.2,0.023892,4.441228,4.465120,0.023745,3.242240,3.265985
3,0.4,0.093225,9.739133,9.832359,0.092878,8.163417,8.256295
4,0.6,0.326080,22.676560,23.002639,0.325349,21.343528,21.668877
5,0.8,1.076302,55.661181,56.737483,1.074859,56.030503,57.105362
6,1.0,3.428035,141.971008,145.399043,3.425305,147.125129,150.550435


__Question 3__

(a)

In [110]:
def ab2(funs, x0, y0, x1, y1, N, h):
    "This is Adams-Bashforth 2-point method"
    
    res = {'x':[x0, x1], 'y':[y0, y1]}

    for i in N:
        y2 = y1 + (3/2)*h*funs(x1, y1) - (h/2)*funs(x0,y0)
        
        res['x'].append(x1+h)
        res['y'].append(y2)
        
        x0 = x1
        y0 = y1
        x1 = x1+h
        y1 = y2
        
    return res

def q3seed(x):
    "this is to get seed values based on homework Q3"
    
    y = (x**2)/2
    
    return y

def q3funs(x, y):
    "this is the ODE in Q3"
    
    y_prime = x+y
    
    return y_prime

In [111]:
steps = range(4)

#seed values for 2-point methods
x_true0 = 0
y_true0 = 0

xseed1 = 0.1
yseed1 = q3seed(xseed1)

In [112]:
ab2(q3funs, x_true0, y_true0, xseed1, yseed1, steps, 0.1)

{'x': [0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5],
 'y': [0,
  0.005000000000000001,
  0.020750000000000005,
  0.0486125,
  0.08986687500000001,
  0.14591628125000003]}

(b)

In [116]:
def am2(x0, y0, x1, y1, steps, h):
    "This is Adams-Moulton 2-step method (implicit) specifically for Q3"
    
    res = {'x':[x0, x1], 'y':[y0, y1]}
    x2 = x1+h
    
    for i in steps:
        
        y2 = (12/(12-5*h))*(y1 + (5*h/12)*x2 + (h/12)*(8*(x1+y1) - (x0 + y0)))
        
        res['x'].append(x2)
        res['y'].append(y2)
        
        x0 = x1
        y0 = y1
        x1 = x2
        y1 = y2  
        x2 = x2+h

    return res

am2(x_true0, y_true0, xseed1, yseed1, range(4), 0.1)

{'x': [0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5],
 'y': [0,
  0.005000000000000001,
  0.021217391304347827,
  0.049659357277882794,
  0.09161026382838826,
  0.1484909079370071]}